In [ ]:
import os
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [ ]:
# === Chargement des variables d'environnement ===
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [ ]:
# === Connexion à la base PostgreSQL RDS ===
try:
    engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    df = pd.read_sql("SELECT * FROM jobs", con=engine)
    print("✅ Données chargées depuis RDS avec succès")
except Exception as e:
    print("❌ Erreur de connexion à la base RDS :", e)
    df = pd.DataFrame()  # On crée un df vide pour éviter de planter la suite

In [ ]:
# Nettoyage de base
if not df.empty:
    df.dropna(subset=["title", "company", "location"], inplace=True)
    df["title"] = df["title"].str.lower().str.strip()
    df["company"] = df["company"].str.strip()
    df["location"] = df["location"].str.strip()

    # Aperçu des données
    print("\nAperçu des données :")
    print(df.head())

In [ ]:
# === Visualisation 1 : Offres par ville ===
df_city = df["location"].value_counts().head(15).reset_index()
df_city.columns = ["city", "count"]
fig1 = px.bar(df_city, x="city", y="count", title="Top 10 des villes avec le plus d'offres")
fig1.show()

In [ ]:
 # === Visualisation 2 : Entreprises qui recrutent le plus ===
df_company = df["company"].value_counts().head(15).reset_index()
df_company.columns = ["company", "count"]
fig2 = px.pie(df_company, names="company", values="count", title="Top entreprises qui recrutent")
fig2.show()

In [ ]:
    # === Visualisation 3 : Distribution des salaires ===
    df_salaire = df.dropna(subset=["salary_min", "salary_max"])
    fig3 = px.box(df_salaire, y="salary_max", points="all", title="Distribution des salaires max")
    fig3.show()

In [ ]:
    # === Visualisation 4 : Offres par source ===
    df_source = df["source"].value_counts().reset_index()
    df_source.columns = ["source", "count"]
    fig4 = px.pie(df_source, names="source", values="count", title="Origine des offres (API vs Scraping)")
    fig4.show()

In [ ]:
    # === Visualisation 5 : Top intitulés de poste ===
    df_title = df["title"].value_counts().head(15).reset_index()
    df_title.columns = ["title", "count"]
    fig5 = px.bar(df_title, x="title", y="count", title="Top intitulés de poste")
    fig5.show()

In [ ]:
    # === Visualisation 7 : Nombre d'offres avec ou sans salaire ===
    df["has_salary"] = df["salary_min"].notnull() & df["salary_max"].notnull()
    salary_counts = df["has_salary"].value_counts().rename({True: "Avec salaire", False: "Sans salaire"})
    fig7 = px.pie(values=salary_counts.values, names=salary_counts.index, title="Présence des salaires dans les offres")
    fig7.show()

In [ ]:
    # === Insight exemple ===
    print("\n🔎 Insight :")
    print("La majorité des offres viennent de :", df["source"].value_counts().idxmax())
    print("Nombre d'offres avec salaire renseigné :", len(df_salaire))


In [ ]:
    # === Insights supplémentaires ===
    print("\n🔎 Insights supplémentaires :")
    print("1. Nombre total d'offres :", len(df))
    print("2. Nombre d'entreprises uniques :", df["company"].nunique())
    print("3. Nombre de villes uniques :", df["location"].nunique())
    print("4. Intitulé de poste le plus fréquent :", df["title"].mode()[0])
    print("5. Entreprise ayant posté le plus d'offres :", df["company"].value_counts().idxmax())
    print("6. Salaire moyen max :", df["salary_max"].dropna().mean())
    print("7. Salaire médian min :", df["salary_min"].dropna().median())
    print("8. Pourcentage d'offres avec salaire renseigné :", round(df["has_salary"].mean() * 100, 2), "%")
    print("9. Nombre d'intitulés de poste uniques :", df["title"].nunique())
    print("10. Répartition API vs Scraping :")
    print(df["source"].value_counts(normalize=True) * 100)


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x="source", palette="Set2")
plt.title("Nombre d'offres par source")
plt.ylabel("Nombre d'offres")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

In [ ]:
top_titles = df["title"].value_counts().head(10)

plt.figure(figsize=(8, 5))
sns.barplot(x=top_titles.values, y=top_titles.index, palette="coolwarm")
plt.title("Top 10 des intitulés de poste")
plt.xlabel("Nombre d'offres")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, y="contract_type", order=df["contract_type"].value_counts().index, palette="pastel")
plt.title("Répartition des types de contrats")
plt.xlabel("Nombre d'offres")
plt.tight_layout()
plt.show()


In [ ]:
print("Statistiques sur salary_min :\n", df["salary_min"].describe())
print("Statistiques sur salary_max :\n", df["salary_max"].describe())


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df["salary_min"], bins=30, color='skyblue', label="Min", kde=True)
sns.histplot(df["salary_max"], bins=30, color='salmon', label="Max", kde=True)
plt.legend()
plt.title("Distribution des salaires")
plt.xlabel("Salaire (€)")
plt.tight_layout()
plt.show()


In [ ]:
top_locations = df["location"].value_counts().head(10)

plt.figure(figsize=(8, 5))
sns.barplot(x=top_locations.values, y=top_locations.index, palette="viridis")
plt.title("Top 10 des localisations")
plt.xlabel("Nombre d'offres")
plt.tight_layout()
plt.show()


In [ ]:
if "category" in df.columns:
    plt.figure(figsize=(8, 5))
    df["category"].value_counts().head(10).plot(kind="barh", color="lightgreen")
    plt.title("Top 10 des catégories d'emploi")
    plt.xlabel("Nombre d'offres")
    plt.tight_layout()
    plt.show()
